In [ ]:
import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

from pandas import Series

In [ ]:
def test_func(inp, test_str):
    if type(inp) != str:
        return False
    else:
        return test_str in inp
    
def sum_up(x):
     return Series(dict(Qty = x['Qty'].sum(), 
                        Size = "{%s}" % ', '.join(x['Size'].astype(str))))

# Load and format data

In [ ]:
df = pd.read_excel('/Users/nathansuberi/Downloads/OH_Items4.xls')

# With settings

df = df[['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 8', 'Unnamed: 13', 'Unnamed: 17', 'Unnamed: 24', 'Unnamed: 27', 'Unnamed: 31']]
df.columns = ['DCS', 'Vend Code', 'Desc1', 'Attr', 'Size', 'Qty', 'Price', 'Cost']
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
df = df.drop(16)
keep_data4 = df

In [ ]:
df = pd.read_excel('/Users/nathansuberi/Downloads/OH_Items2.xls')

# For Items2, By Item by Subsidiary
df = df.drop(['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 3'], axis=1)
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
df = df[['Unnamed: 7', 'ON HAND: by Item By Subsidiary', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 16', 'Unnamed: 23', 'Unnamed: 25', 'Report Date:']]
df.columns = ['DCS', 'Vend Code', 'Desc1', 'Attr', 'Size', 'Qty', 'Price', 'Cost']
df = df.drop([1, 14, 15])

keep_data = df[['DCS', 'Vend Code', 'Desc1', 'Attr', 'Size']]
qty_data = df[['Qty', 'Price', 'Cost']]
keep_data = keep_data.dropna(axis=0, how='all')
keep_data2 = keep_data.join(qty_data)

In [ ]:
df = pd.read_excel('/Users/nathansuberi/Downloads/OH_Items3.xls')

# For Items3, By Item
df = df.drop(['Unnamed: 0', 'Unnamed: 2'], axis=1)
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
df = df[['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 8', 'Unnamed: 13', 'Unnamed: 17', 'Unnamed: 24', 'Unnamed: 27', 'Unnamed: 31']]
df.columns = ['DCS', 'Vend Code', 'Desc1', 'Attr', 'Size', 'Qty', 'Price', 'Cost']
keep_data3 = df.drop([0, 2, 11])

In [ ]:
df = keep_data2
col = 'Qty'

flag = [a and b for a,b in zip(df[col].notnull(), df[col]>0)]
df.loc[flag, col].sum()

In [ ]:
df[df[col]>0]

# Pull just Merrells to check against Inventory View

In [ ]:
df = keep_data4

mer_flag = [test_func(vnd, 'MER') for vnd in df['Vend Code']]
merrells = df[mer_flag]

In [ ]:
merrells[merrells['Desc1'].str.contains('MIMOSA')]

# Designate small sums

In [ ]:
sums = df.groupby(['DCS', 'Vend Code', 'Desc1', 'Attr']).sum()
sums = sums.reset_index()
inventory = sums[sums['Qty'] > 0]
small_inv = inventory[inventory['Qty'] <= 2]
small_inv.to_html('/Users/nathansuberi/Desktop/enchanted_reports/Small_inventory.html')

In [ ]:
vendors = sums['Vend Code'].unique()

In [ ]:
vendors

In [ ]:
print_data = df.sort_values(['Vend Code', 'Desc1', 'Attr', 'Size'])
print_data = print_data[print_data['Qty'] > 0]
print_data = print_data.groupby(['Vend Code', 'Desc1', 'Attr']).apply(sum_up).reset_index()

In [ ]:
with open('/Users/nathansuberi/Desktop/enchanted_reports/inventory_analysis/inventory_reference.html', 'w') as f:
    for vendor in vendors:
        vendor_data = print_data[print_data['Vend Code'] == vendor]
        vendor_data['key'] = vendor_data['Desc1'] + vendor_data['Attr']
        
        try:
            vendor_not_on_sale = pd.read_csv('/Users/nathansuberi/Desktop/enchanted_reports/summer_sale_2018/notonsale_{}.csv'.format(vendor))
            vendor_not_on_sale.columns = ['index', 'Desc1', 'Attr', 'SOLD']
            vendor_not_on_sale['key'] = vendor_not_on_sale['Desc1'] + vendor_not_on_sale['Attr']
            not_on_sales = True
        except:
            not_on_sales = False
        
#         print("hello")
#         print(vendor_data.head())
#         print("world")
#         print(vendor_not_on_sale.head())
        
        f.write('<h1>Vendor: ' + vendor + '</h1>\n')
        last_style = ''
        for line in vendor_data.values:
            style = line[1]
            color = line[2]
            sizes = line[4].replace('{', '').replace('}', '')
            if last_style == style:
                header = '</br><i>'
                footer = '</i>'
            else:
                header = '<br/><b>'
                footer = '</b>'
                
            if not_on_sales:                     
                if line[5] not in vendor_not_on_sale['key'].values:
                    header = '<mark>' + header
                    footer = footer + '</mark>'
                
            f.write('{}{}, {}, sizes: <font size=4>{}</font>{}'.format(header, style, color, sizes, footer))
            last_style = style

In [ ]:
keep_line[5] in keep_vend.values